In [9]:
import numpy as np
import wave
import pywt
import scipy.signal as signal
from scipy.fftpack import dct
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
import os

def enframe(signal, nw, inc,winfunc):
    '''将音频信号转化为帧。
    参数含义：
    signal:原始音频型号
    nw:每一帧的长度(这里指采样点的长度，即采样频率乘以时间间隔)
    inc:相邻帧的间隔（同上定义）
    '''
    signal_length = len(signal)  # 信号总长度
    if signal_length <= nw:  # 若信号长度小于一个帧的长度，则帧数定义为1
        nf = 1
    else:  # 否则，计算帧的总长度
        nf = int(np.ceil((1.0 * signal_length - nw + inc) / inc))
    pad_length = int((nf - 1) * inc + nw)  # 所有帧加起来总的铺平后的长度
    zeros = np.zeros((pad_length - signal_length,))  # 不够的长度使用0填补，类似于FFT中的扩充数组操作
    pad_signal = np.concatenate((signal, zeros))  # 填补后的信号记为pad_signal
    indices = np.tile(np.arange(0, nw), (nf, 1)) + np.tile(np.arange(0, nf * inc, inc),
                                                           (nw, 1)).T  # 相当于对所有帧的时间点进行抽取，得到nf*nw长度的矩阵
    indices = np.array(indices, dtype=np.int32)  # 将indices转化为矩阵
    frames = pad_signal[indices]  # 得到帧信号
    win=np.tile(winfunc,(nf,1))  #window窗函数，这里默认取1
    return frames*win   #返回帧信号矩阵


def wavread(filename):
    f = wave.open(filename, 'rb')
    params = f.getparams()
    nchannels, sampwidth, framerate, nframes = params[:4]
    strData = f.readframes(nframes)  # 读取音频，字符串格式
    waveData = np.frombuffer(strData, dtype=np.int16)  # 将字符串转化为int
    f.close()
    waveData = waveData * 1.0 / (max(abs(waveData)))  # wave幅值归一化
    waveData = np.reshape(waveData, [nframes, nchannels]).T
    return framerate,waveData

def DCT(Frame):
    return dct(Frame[0])


def DFT(Frame):
    real = []
    image = []
    result = []
    for i in range(0,1024):
        real.append(0)
        image.append(0)
        result.append(0)
    for seg in Frame:
        N = len(seg)
        for k in range(N):
            for n in range(N):
                real[k] = real[k]+seg[n]*np.cos(2*np.pi*k*n/N)
                image[k] = image[k]-seg[n]*np.sin(2*np.pi*k*n/N)
        for i in range(N):
            result[i] = np.sqrt(real[i]*real[i]+image[i]*image[i])
        return result



if __name__ == '__main__':
    filename = "Ring10.wav"  # 添加路径
    framerate,data = wavread(filename)
    nw = 1024
    inc = 128
    winfunc = signal.hamming(nw)
    Frame = enframe(data[0], nw, inc, winfunc)

    time = np.arange(0, nw) * (1.0 / framerate)
    a = Frame[1:2]
    plt.figure(figsize=(5, 7))
    plt.subplot(511)
    plt.plot(time, Frame[0] , c="g")

    fft1 = np.fft.fft(Frame[0])
    plt.subplot(512)
    plt.plot(time, fft1, c="g")

    # fft3 = []
    # fft2 = DFT(Frame)
    # plt.subplot(413)
    # for i in fft2:
    #     fft3.append(i*(-1))
    # plt.plot(time, fft2, c="g")
    # plt.plot(time, fft3, c="g")

    dct1 = DCT(Frame)
    plt.subplot(513)
    plt.plot(time , dct1 , c="g")

#     (cA,cD) = pywt.dwt(Frame[0],"db1")
    cA, cD = pywt.dwt(Frame[0], 'db2', 'smooth')
    upcoef_cA = pywt.upcoef('a', cA, 'db2', take=1024)
    upcoef_cD = pywt.upcoef('d', cD, 'db2', take=1024)
    res = upcoef_cA + upcoef_cD
    plt.subplot(514)
    plt.plot(time, upcoef_cA, c="g")
    plt.grid()
    plt.subplot(515)
    plt.plot(time, upcoef_cD, c="g")
    plt.grid()
    plt.show()


c:\users\yi\appdata\local\programs\python\python37\lib\site-packages\numpy\core\_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
